# Setup

In [1]:
import sys

sys.path.append("..")

In [2]:
# Main Constants
from pathlib import Path

# Base path for testing - adjust as needed
BASE_PATH = Path("temp").resolve()  # Points to anthropic_agent root
print(BASE_PATH)

/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/anthropic_agent/common_tools/temp


# Tool Tests

## Tool: Read File

In [3]:
from anthropic_agent.common_tools.read_file import ReadFileTool

# Create the read file tool
read_file_tool = ReadFileTool(base_path=BASE_PATH)
read_file = read_file_tool.get_tool()

In [4]:
read_file("README.md")

'[lines 0-0 of 0 in README.md]\n'

## Tool: List Dir

In [5]:
from anthropic_agent.common_tools.list_dir import ListDirTool

list_dir_tool = ListDirTool(base_path=BASE_PATH)
list_dir = list_dir_tool.get_tool()

In [6]:
list_dir(".")

'temp/\n   - README.md'

## Tool: Glob File Search

In [7]:
from anthropic_agent.common_tools.glob_file_search import GlobFileSearchTool

glob_search_tool = GlobFileSearchTool(base_path=BASE_PATH)
glob_file_search = glob_search_tool.get_tool()

In [8]:
glob_file_search("*.md")

'README.md'

## Tool: Grep Search

In [9]:
from anthropic_agent.common_tools.grep_search import GrepSearchTool

grep_search_tool = GrepSearchTool(base_path=BASE_PATH)
grep_search = grep_search_tool.get_tool()

In [10]:
grep_search("README")

'No matches found.'

## Tool: Apply Patch

In [11]:
from anthropic_agent.common_tools.apply_patch import ApplyPatchTool

apply_patch_tool = ApplyPatchTool(base_path=BASE_PATH)
apply_patch = apply_patch_tool.get_tool()

In [13]:
# Dry run example - won't actually modify files
patch = """*** Begin Patch
*** Add File: test_file.md
+# Test File
+This is a test.
*** End Patch"""

apply_patch(patch, dry_run=False)

'{"status": "ok", "op": "add", "path": "test_file.md", "hunks_applied": 0, "lines_added": 2, "lines_removed": 0, "dry_run": false}'